In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gp

In [2]:
import os
import json
wd = os.getcwd()
wd_list = wd.split("/")
unique_path = "/".join(wd_list[1:3])
#I have the passwords file saved as 'nothing_to_see_here.json'. Super secure, I know
path = "/"+unique_path+"/esh/nothing_to_see_here.json"

#Load json file in .esh and get into creds
#This accesses for the db_creds dictionary specifically. Other dicts are available by changing the ref below
creds = json.load(open(path))['db_creds']
from sqlalchemy import create_engine

###DONT SHARE WITH LOGIN DETAILS###
POSTGRES_ADDRESS = creds['esh_db_address']
POSTGRES_PORT =  creds['esh_db_port']
POSTGRES_USERNAME = creds['esh_db_username']
POSTGRES_PASSWORD = creds['esh_db_password']
POSTGRES_DBNAME = creds['esh_db_name']
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
                password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

JSONDecodeError: Expecting value: line 15 column 2 (char 546)

In [3]:
query = '''
WITH df as (
	SELECT 
		--identifying info
		hh.state_id, hh.puma20_id, hh.serialno, 
		-- eligibility
		MAX(
			CASE WHEN (pop.hins4::text = '1'::text OR 
									hh.fs::text = '1'::text OR
									pop.pap > 0::numeric OR
									pop.ssip > 0::numeric OR 
									pop.povpip <= 200::numeric) 
			THEN 1
			ELSE 0 END) as pbb_eligible,
		--unconnected
		MAX(
			CASE WHEN  hh.accessinet::text = '3'::text OR
						(hh.accessinet::text = '1'::text AND 
						 hh.hispeed::text = '2'::text AND 
						 hh.othsvcex::text = '2'::text AND 
						 (hh.dialup::text = '1'::text OR hh.dialup::text = '2'::text) AND 
						 hh.satellite::text = '2'::text) 
			THEN 1
			ELSE 0 END) AS unconnected,
		--subscription vulnerable method
		MAX(
			CASE WHEN hh.hincp <= 50000
			THEN 1
			ELSE 0 END) as sub_50k,
		MAX(hh.wgtp) as hh_weight
		FROM dl.pums_households_2022 hh
		LEFT JOIN dl.pums_population_2022 pop ON hh.puma20_id::text = pop.puma20_id::text AND hh.serialno::text = pop.serialno::text
		WHERE hh.wgtp>0
		GROUP BY hh.state_id, hh.serialno, hh.puma20_id, 
			hh.accessinet, hh.hispeed, hh.othsvcex, hh.dialup, hh.satellite
)		
SELECT state_id,

SUM(CASE WHEN unconnected = 1 
	THEN hh_weight
	ELSE 0 END) AS unconnected, 	
	
SUM(CASE WHEN pbb_eligible = 1 
	THEN hh_weight
	ELSE 0 END) AS pbb_eligible,

SUM(CASE WHEN pbb_eligible = 1 AND unconnected = 1 
	THEN hh_weight
	ELSE 0 END) AS pbb_eligi_unconnected

FROM df
GROUP BY state_id
'''

In [4]:
df = pd.read_sql(query, cnx)
df.head()

,state_id,unconnected,pbb_eligible,pbb_eligi_unconnected
0,01,566705.0,907340.0,355108.0
1,02,67260.0,101493.0,36491.0
2,04,538640.0,1175293.0,332124.0
3,05,367410.0,605148.0,238051.0
4,06,2300227.0,5989677.0,1461144.0


In [5]:
states_query = '''
SELECT state_id, geo_name as state_name, total_households
FROM ps.states
WHERE year = (SELECT MAX(year) FROM ps.states)
'''
states = pd.read_sql(states_query, cnx)
states.head()

,state_id,state_name,total_households
0,01,Alabama,1888504.0
1,02,Alaska,255173.0
2,04,Arizona,2643430.0
3,05,Arkansas,1170544.0
4,06,California,13103114.0


In [6]:
df = df.merge(states, how = 'left', on='state_id')
df.head()

,state_id,unconnected,pbb_eligible,pbb_eligi_unconnected,state_name,total_households
0,01,566705.0,907340.0,355108.0,Alabama,1888504.0
1,02,67260.0,101493.0,36491.0,Alaska,255173.0
2,04,538640.0,1175293.0,332124.0,Arizona,2643430.0
3,05,367410.0,605148.0,238051.0,Arkansas,1170544.0
4,06,2300227.0,5989677.0,1461144.0,California,13103114.0


In [7]:
df.shape

(52, 6)

In [13]:
import plotly.express as px
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

# State abbreviation mapping
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Add a column for state abbreviations
df['state_abbrev'] = df['state_name'].map(state_abbrev)

# Create the choropleth map
fig = px.choropleth(df,
                    locations='state_abbrev',
                    locationmode="USA-states",
                    color='pbb_eligi_unconnected',
                    scope="usa",
                    color_continuous_scale="YlOrRd",
                    labels={'unconnected':'Households'},
                    title='PBB Eligible Unconnected Households by State'
                   )

fig.update_layout(geo=dict(bgcolor= 'rgba(0,0,0,0)'))
fig.update_traces(marker_line_color = 'white', selector=dict(type='choropleth'))

fig.show()


In [9]:
# Create the choropleth map
fig_2 = px.choropleth(df,
                    locations='state_abbrev',
                    locationmode="USA-states",
                    color='pbb_eligi_unconnected',
                    scope="usa",
                    color_continuous_scale="Greens",
                    labels={'pbb_eligi_unconnected':'PBB Eligible Households'},
                    title='PBB Eligible (Unconnected) Households by State'
                   )

fig_2.update_layout(geo=dict(bgcolor= 'rgba(0,0,0,0)'))
fig_2.show()

In [19]:
df['pct_elig_unc'] = df['pbb_eligi_unconnected']/df['total_households']
df_nopr = df.drop(index=51,axis=0)
#create heatmap
fig_3 = px.choropleth(df_nopr,
                    locations='state_abbrev',
                    locationmode="USA-states",
                    color='pct_elig_unc',
                    scope="usa",
                    color_continuous_scale="Blues",
                    labels={'pct_elig_unc':'Percent of Households'},
                    title='Percent of Households PBB Eligible (Unconnected) Households by State'
                   )

fig_3.update_layout(geo=dict(bgcolor= 'rgba(0,0,0,0)'))

fig_3.show()

In [14]:
df_nopr.sort_values('pct_elig_unc').tail()

,state_id,unconnected,pbb_eligible,pbb_eligi_unconnected,state_name,total_households,state_abbrev,pct_elig_unc
0,01,566705.0,907340.0,355108.0,Alabama,1888504.0,AL,0.188037
31,35,222678.0,431190.0,154149.0,New Mexico,792755.0,NM,0.194447
18,22,512803.0,930388.0,354846.0,Louisiana,1751956.0,LA,0.202543
3,05,367410.0,605148.0,238051.0,Arkansas,1170544.0,AR,0.203368
24,28,401897.0,595763.0,266195.0,Mississippi,1116649.0,MS,0.238387
